In [9]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
# Export model
def export_onnx(model, data_tensor_array, model_loc):
  circuit = model()

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  # print(device)

  circuit.to(device)

  # Flips the neural net into inference mode
  circuit.eval()
  input_names = []
  dynamic_axes = {}

  data_tensor_tuple = ()
  for i in range(len(data_tensor_array)):
    data_tensor_tuple += (data_tensor_array[i],)
    input_index = "input"+str(i+1)
    input_names.append(input_index)
    dynamic_axes[input_index] = {0 : 'batch_size'}
  dynamic_axes["output"] = {0 : 'batch_size'}

  # Export the model
  torch.onnx.export(circuit,               # model being run
                      data_tensor_tuple,                   # model input (or a tuple for multiple inputs)
                      model_loc,            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=11,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = input_names,   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes=dynamic_axes)

In [3]:
# mode is either "accuracy" or "resources"

def gen_settings(comb_data_path, onnx_filename, scale, mode, settings_filename):
  print("==== Generate & Calibrate Setting ====")
  # Set input to be Poseidon Hash, and param of computation graph to be public
  # Poseidon is not homomorphic additive, maybe consider Pedersens or Dory commitment.
  gip_run_args = ezkl.PyRunArgs()
  gip_run_args.input_visibility = "hashed"  # matrix and generalized inverse commitments
  gip_run_args.output_visibility = "public"   # no parameters used
  gip_run_args.param_visibility = "private" # should be Tensor(True)--> to enforce arbitrary data in w

 # generate settings
  ezkl.gen_settings(onnx_filename, settings_filename, py_run_args=gip_run_args)
  if scale =="default":
    ezkl.calibrate_settings(
    comb_data_path, onnx_filename, settings_filename, mode)
  else:
    ezkl.calibrate_settings(
    comb_data_path, onnx_filename, settings_filename, mode, scales = scale)

  assert os.path.exists(settings_filename)
  assert os.path.exists(comb_data_path)
  assert os.path.exists(onnx_filename)
  f_setting = open(settings_filename, "r")
  print("scale: ", scale)
  print("setting: ", f_setting.read())


In [4]:
def verifier_init(verifier_model, verifier_model_path, verifier_compiled_model_path, dummy_data_path_array, settings_path, srs_path, pk_path, vk_path, scale, mode):

  # load data from dummy_data_path_array into dummy_data_array
  dummy_data_tensor_array = []
  comb_dummy_data = []
  for path in dummy_data_path_array:
    dummy_data = np.array(json.loads(open(path, "r").read())["input_data"][0])
    # print("dumm: ", dummy_data)
    dummy_data_tensor_array.append(torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 )))
    comb_dummy_data.append(dummy_data.tolist())
  # export onnx file
  export_onnx(verifier_model,dummy_data_tensor_array, verifier_model_path)

  comb_dummy_data_path = os.path.join('generated/comb_dummy_data.json')
  # Serialize data into file:
  json.dump(dict(input_data = comb_dummy_data), open(comb_dummy_data_path, 'w' ))

  # gen + calibrate setting
  gen_settings(comb_dummy_data_path, verifier_model_path, scale, mode, settings_path)

  # compile circuit
  res = ezkl.compile_circuit(verifier_model_path, verifier_compiled_model_path, settings_path)
  assert res == True

  # srs path
  res = ezkl.get_srs(srs_path, settings_path)

  # setupt vk, pk param for use..... prover can use same pk or can init their own!
  print("==== setting up ezkl ====")
  start_time = time.time()
  res = ezkl.setup(
        verifier_compiled_model_path,
        vk_path,
        pk_path,
        srs_path)
  end_time = time.time()
  time_setup = end_time -start_time
  print(f"Time setup: {time_setup} seconds")

  assert res == True
  assert os.path.isfile(vk_path)
  assert os.path.isfile(pk_path)
  assert os.path.isfile(settings_path)

In [5]:
def prover_gen(prover_model, data_path_array, witness_path, prover_model_path, prover_compiled_model_path, settings_path, proof_path):
  # load data from data_path
  # data = json.loads(open(data_path, "r").read())["input_data"][0]
  # data_tensor = torch.reshape(torch.tensor(data), (1, len(data),1 ))


  data_tensor_array = []
  comb_data = []
  for path in data_path_array:
    data = np.array(json.loads(open(path, "r").read())["input_data"][0])
    # print("dumm: ", dummy_data)
    data_tensor_array.append(torch.reshape(torch.tensor(data), (1, len(data),1 )))
    comb_data.append(data.tolist())

  # export onnx file
  export_onnx(prover_model, data_tensor_array, prover_model_path)

  comb_data_path = os.path.join('generated/comb_data.json')
  # Serialize data into file:
  json.dump(dict(input_data = comb_data), open(comb_data_path, 'w' ))

  res = ezkl.compile_circuit(prover_model_path, prover_compiled_model_path, settings_path)
  assert res == True
  # now generate the witness file
  print('==== Generating Witness ====')
  witness = ezkl.gen_witness(comb_data_path, prover_compiled_model_path, witness_path)
  assert os.path.isfile(witness_path)
  # print(witness["outputs"])
  settings = json.load(open(settings_path))
  output_scale = settings['model_output_scales']
  print("witness boolean: ", ezkl.vecu64_to_float(witness['outputs'][0][0], output_scale[0]))
  for i in range(len(witness['outputs'][1])):
    print("witness result", i+1,":", ezkl.vecu64_to_float(witness['outputs'][1][i], output_scale[1]))

  # GENERATE A PROOF
  print("==== Generating Proof ====")
  start_time = time.time()
  res = ezkl.prove(
        witness_path,
        prover_compiled_model_path,
        pk_path,
        proof_path,
        srs_path,
        "single",
    )

  print("proof: " ,res)
  end_time = time.time()
  time_gen_prf = end_time -start_time
  print(f"Time gen prf: {time_gen_prf} seconds")
  assert os.path.isfile(proof_path)

In [6]:
def verifier_verify(proof_path, settings_path, vk_path, srs_path):
  # enforce boolean statement to be true
  settings = json.load(open(settings_path))
  output_scale = settings['model_output_scales']

  proof = json.load(open(proof_path))
  num_inputs = len(settings['model_input_scales'])
  print("num_inputs: ", num_inputs)
  proof["instances"][0][num_inputs] = ezkl.float_to_vecu64(1.0, output_scale[0])
  json.dump(proof, open(proof_path, 'w'))

  print("prf instances: ", proof['instances'])

  print("proof boolean: ", ezkl.vecu64_to_float(proof['instances'][0][num_inputs], output_scale[0]))
  for i in range(num_inputs+1, len(proof['instances'][0])):
    print("proof result",i-num_inputs,":", ezkl.vecu64_to_float(proof['instances'][0][i], output_scale[1]))


  res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        srs_path,
    )

  assert res == True
  print("verified")

In [7]:
# init path
os.makedirs(os.path.dirname('generated/'), exist_ok=True)
verifier_model_path = os.path.join('generated/verifier.onnx')
prover_model_path = os.path.join('generated/prover.onnx')
verifier_compiled_model_path = os.path.join('generated/verifier.compiled')
prover_compiled_model_path = os.path.join('generated/prover.compiled')
pk_path = os.path.join('generated/test.pk')
vk_path = os.path.join('generated/test.vk')
proof_path = os.path.join('generated/test.pf')
settings_path = os.path.join('generated/settings.json')
srs_path = os.path.join('generated/kzg.srs')
witness_path = os.path.join('generated/witness.json')

=======================  ZK-STATS FLOW =======================

In [9]:
x1_vals_path = os.path.join('x1_vals.json')
x2_vals_path = os.path.join('x2_vals.json')
dummy_x1_vals_path = os.path.join('generated/dummy_x1_vals.json')
dummy_x2_vals_path = os.path.join('generated/dummy_x2_vals.json')

x1_vals= np.array(json.loads(open(x1_vals_path, "r").read())['input_data'][0])
dummy_x1_vals = np.random.uniform(min(x1_vals), max(x1_vals), len(x1_vals))
json.dump({"input_data":[dummy_x1_vals.tolist()]}, open(dummy_x1_vals_path, 'w'))

x2_vals= np.array(json.loads(open(x2_vals_path, "r").read())['input_data'][0])
dummy_x2_vals = np.random.uniform(min(x2_vals), max(x2_vals), len(x2_vals))
json.dump({"input_data":[dummy_x2_vals.tolist()]}, open(dummy_x2_vals_path, 'w'))


y_vals_path = os.path.join('y_vals.json')
dummy_y_vals_path = os.path.join('generated/dummy_y_vals.json')

y_vals= np.array(json.loads(open(y_vals_path, "r").read())['input_data'][0])
dummy_y_vals = np.random.uniform(min(y_vals), max(y_vals), len(y_vals))
json.dump({"input_data":[dummy_y_vals.tolist()]}, open(dummy_y_vals_path, 'w'))


def stacked_x(*args):
    result = np.column_stack((*args, np.ones_like(args[0])))
    return result

x_one = stacked_x(x1_vals, x2_vals)
dummy_x_one = stacked_x(dummy_x1_vals, dummy_x2_vals)

w_vals = np.matmul(np.matmul(np.linalg.inv(np.matmul(x_one.transpose(), x_one)), x_one.transpose()), y_vals)
dummy_w_vals = np.matmul(np.matmul(np.linalg.inv(np.matmul(dummy_x_one.transpose(), dummy_x_one)), dummy_x_one.transpose()), dummy_y_vals)

print("reg fit: ", w_vals)
w_tensor = torch.reshape(torch.tensor(w_vals), (1,3,1))
dummy_w_tensor = torch.reshape(torch.tensor(dummy_w_vals), (1,3,1))


reg fit:  [1.98835287 3.14238058 0.06413647]


In [18]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        self.w = nn.Parameter(data = dummy_w_tensor, requires_grad = False)

    def forward(self, *args):
        # infer Y from the last parameter
        Y = args[-1]
        X_one = torch.cat((*args[:-1], torch.ones_like(args[0][:, :, -1:])), dim=2)
        X_T = torch.transpose(X_one, 1, 2)
        # some expression of tolerance to error in the inference
        return (
            torch.sum(torch.abs(X_T @ X_one @ self.w.to('cpu') - X_T @ Y)) < 0.01 * torch.sum(X_T @ Y),
            self.w.to('cpu')
        )
verifier_init(verifier_model, verifier_model_path, verifier_compiled_model_path, [dummy_x1_vals_path,dummy_x2_vals_path, dummy_y_vals_path], settings_path, srs_path, pk_path, vk_path,[3], "resources")

==== Generate & Calibrate Setting ====
scale:  [3]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":3,"param_scale":3,"scale_rebase_multiplier":10,"lookup_range":[-13942,977004],"logrows":20,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":11808,"total_assignments":784,"total_const_size":4,"model_instance_shapes":[[1],[1,3,1]],"model_output_scales":[0,3],"model_input_scales":[3,3,3],"module_sizes":{"kzg":[],"poseidon":[11808,[3]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Div":{"denom":100.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 88.79235982894897 seconds


In [19]:
# Prover/ data owner side
theory_output = w_tensor
print("theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        self.w = nn.Parameter(data = w_tensor, requires_grad = False)

    def forward(self, *args):
        # infer Y from the last parameter
        Y = args[-1]
        X_one = torch.cat((*args[:-1], torch.ones_like(args[0][:, :, -1:])), dim=2)
        X_T = torch.transpose(X_one, 1, 2)
        # some expression of tolerance to error in the inference
        return (
            torch.sum(torch.abs(X_T @ X_one @ self.w.to('cpu') - X_T @ Y)) < 0.01 * torch.sum(X_T @ Y),
            self.w.to('cpu')
        )
prover_gen(prover_model, [x1_vals_path, x2_vals_path, y_vals_path], witness_path, prover_model_path, prover_compiled_model_path, settings_path, proof_path)

==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 2.0
witness result 2 : 3.125
witness result 3 : 0.125
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[13372645934914757851, 4179548409246906955, 7308753190278705352, 585334253871252673], [14296875065875978922, 10817153402807448554, 16646795345921275234, 1848850777343635446], [11033437535517255964, 15518297366369924990, 13029218448037200835, 1837434524902037587], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [13392416068245389228, 14913608834158628949, 9631264589208289646, 2240050767903289935], [18409976096712818556, 4988926951396865304, 17770093507857248247, 884830624746662524], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287]]], 'proof': '187847a2ac59f58ce208f8ba0f81fb73bc1e03b076584934c7de4acaf113d36505d04e12614aa78d282867f8a81f9ad074af0b4ae6446e830f42f253a974dd8b0c72ff48553c22bc5d34ff98710b652e23eca2e21b80594e97e91a37db848976139bfe818c8ace2f54efbb9baf1b4f4a28b7877ee4a96b807ad7ccb7e472e4ba0aa5a7e5ef7bf2da4c74a7c54ffb24d461b2241231c00ede224a8fe752ee067d1c885029633a29c89e87ad7b670dd9d

In [21]:
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  3
prf instances:  [[[13372645934914757851, 4179548409246906955, 7308753190278705352, 585334253871252673], [14296875065875978922, 10817153402807448554, 16646795345921275234, 1848850777343635446], [11033437535517255964, 15518297366369924990, 13029218448037200835, 1837434524902037587], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [13392416068245389228, 14913608834158628949, 9631264589208289646, 2240050767903289935], [18409976096712818556, 4988926951396865304, 17770093507857248247, 884830624746662524], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287]]]
proof boolean:  1.0
proof result 1 : 2.0
proof result 2 : 3.125
proof result 3 : 0.125
verified
